<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Libraries: icoscp_core

This example uses a foundational library called `icoscp_core` which can be used to access time-series ICOS data that are <i>previewable</i> in the ICOS Data Portal. "Previewable" means that it is possible to visualize the data variables in the preview plot. The library can also be used to access (meta-)data from [ICOS Cities](https://citydata.icos-cp.eu/portal/) and [SITES](https://data.fieldsites.se/portal/) data repositories. 

Documentation of the library, including information on running it locally, can be found on [PyPI.org](https://pypi.org/project/icoscp_core/).

# Example: Access and work with ocean data

More examples for how to work with ocean data can be found in "ex3_multisource".

### Import libraries

In [ ]:
from icoscp_core.icos import data, meta, ATMO_STATION, ECO_STATION, OCEAN_STATION
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


### List stations

In [ ]:
# Stations specific for the ocean domain (see example 1a and 1b for examples for the atmosphere and ecosystem domains) 
stations = meta.list_stations(OCEAN_STATION)

# Filter stations by country (e.g., Germany, 'DE')
country_code = 'DE'
filtered_stations = [
    s for s in stations
    if s.country_code == country_code
]

# Display available stations in selected country for the atmospheric domain
print("Available stations:")
for station in filtered_stations:
    print(f"Station ID: {station.id}, Name: {station.name}, URI: {station.uri}")


### View metadata for a selected station 

The example shows how to access some of the metadata associated with the station. 

In [ ]:
# Specify a station uri from list above
station_uri = 'http://meta.icos-cp.eu/resources/stations/OS_BALTIC-VOS'
station_meta = meta.get_station_meta(station_uri)

# Print the station name
print('Name:', station_meta.org.name)

print('Staff:')
# Loop over all staff members and print their first name, last name, and email
for staff_member in station_meta.staff:
    first_name = staff_member.person.firstName
    last_name = staff_member.person.lastName
    email = staff_member.person.email
    print(f"{first_name} {last_name} ({email})")


### See what data types are available

In the example, filters are applied so that only data types associated with ICOS Level 2 data that are previewable are shown. See more information [about data levels](https://www.icos-sweden.se/data_availability) here. Additional filters can be applied. Please refer to the documentation for more details.

Note that not all data products are available for all stations. 


In [ ]:
# Available datatypes
data_types = meta.list_datatypes()

# data types with data access (possible to view with Python)
# data types can also be filtered to only include, in this case, level 2 data
data_level = 2
previewable_datatypes = [
    dt for dt in data_types
    if dt.has_data_access and dt.data_level==data_level
]

for data_type in previewable_datatypes:
    datatype_uri = data_type.uri
    datatype_label = data_type.label
    
    print(f"{datatype_label} ({datatype_uri})")
    

### Find data objects based on the selected station and a specified data type

This example shows how to get a list of data objects associated with the selected station and the data type "ICOS OTC SOOP Release". 

In [ ]:
# Specify a data type from the list above 
data_type = 'http://meta.icos-cp.eu/resources/cpmeta/icosOtcL2Product'

station_objects = meta.list_data_objects(datatype = data_type, 
                                         station = station_uri)

i = 0
for station_object in station_objects:
    station_objects_filename = station_object.filename
    
    print(f'Index {i} to access: {station_objects_filename}')
    i = i + 1

if i == 0:
    print(f'No available objects with data type {data_type} at station {station_uri}')

### Access data for a single data object 

This example shows how to access the data and meta data of the object at index 0 in the list above (34FM20231206.csv).

In [ ]:
# Specify the index from the list above
index = 0

if index > len(station_objects) - 1:
    print('Provided index is out of range.')

else:
    # Access metadata associated with the object
    dobj_meta = meta.get_dobj_meta(station_objects[index])

    # Access the object's data
    dobj_arrays = data.get_columns_as_arrays(dobj_meta)

    # Convert to a pandas dataframe
    df = pd.DataFrame(dobj_arrays)

    display(df)

#### Download data to csv-file

Un-comment (remove #) and change the name to save the data shown in the output above.

The output file will be in the same folder as this notebook. 

In [ ]:
#name = 'name_of_file'
#df.to_csv(name + '.csv', encoding = 'utf-8-sig')

### Make a plot: single data column

The selected data_object that has been accessed contains data for GPP. It can be calculated in different ways, but in the example we use the GPP stored in the "GPP_DT_VUT_REF" column in the DataFrame above. If you access a different data object, the data may be stored in a column with a different name. Additionally, the name of the column containing the observation timestamp may also differ.

See more examples of how to plot ocean data in "ex3_multisource". 

In [ ]:
# Run this to see available all columns
df.columns

In [ ]:
time_column = 'TIMESTAMP'
data_column = 'Temp [degC]'

# Find the latest year based on the time_column
latest_year = df[time_column].dt.year.max()

columns_meta = dobj_meta.specificInfo.columns

if data_column in df.columns and time_column in df.columns:

    # find meta data associated with the selected column (data_column)
    dobj_value_type = [col for col in columns_meta if col.label==data_column][0].valueType

    # create label for y-axis based on the meta data
    y_axis_label = f"{dobj_value_type.self.label} [{dobj_value_type.unit}]"
    station = dobj_meta.specificInfo.acquisition.station.org.name

    plot = df.plot(x=time_column, y=data_column, grid=True, title=station, style='o')
    plot.set(ylabel=y_axis_label)

else:
    print(f'The selected data_column or time_column is not one of the columns in df. Choose among {list(df.columns)}.' )

### Combine data for all data objects given the station and data type

Combination of selected data columns from objects in the list "station_objects".

All objects will be considered, but will only be added if they have data stored in columns with the names listed in "data_columns". 

This example looks different compared to the examples for the atmosphere (ex1a) and ocean (ex2a) domains. The reason is that the data objects for a specific data type and station usually have files that correspond to different time periods. These should be concatenated and not merged. Concatenation in this case means that all the rows are just appended and if new columns are introduced by a data object, NaN values are assigned for the rows that miss this new column. 


In [ ]:
time_column = 'TIMESTAMP'

# in the example list of station_objects, there are many more additional columns. Have a look at the output and add columns as needed.
data_columns = ['Temp [degC]', 'pCO2 [uatm]']

available_data_columns = []
y_axis_labels = []

first = True
for dobj, arrs in data.batch_get_columns_as_arrays(station_objects):

    # Convert the arrays into a DataFrame
    df = pd.DataFrame(arrs)

    if time_column not in df.columns:
        print(f"The column given for time ('{time_column}') is not found in {dobj.filename}. Skipping this object.")
        print(f"Available columnns are '{list(df.columns)}'")
        continue
        
    # see which of the desired data_columns are available.
    # save metadata for these.
    found_columns = []
    for data_column in data_columns:
        
        if data_column in df.columns:
            found_columns.append(data_column)

            if data_column not in available_data_columns:

                available_data_columns.append(data_column)

                # find y-axis label for column (used in graph)
                dobj_meta = meta.get_dobj_meta(dobj)
                columns_meta = dobj_meta.specificInfo.columns
                dobj_value_type = [col for col in columns_meta if col.label==data_column][0].valueType
                y_axis_labels.append(f"{dobj_value_type.self.label} [{dobj_value_type.unit}]")
                
    df = df[[time_column] + found_columns]

    if len(found_columns) > 0:
        
        if first:
            final_df = df
            first = False
        else:
            final_df = pd.concat([final_df, df], axis=0)
            
    else:
            
        print(f"None of '{data_columns}' found for {dobj.filename}. Skipping this object.")
        print(f"Available columns are '{list(df_latest_year.columns)}'")

# time_column should be in datetime format. If not alrady, it will be convert to it here:
final_df[time_column] = pd.to_datetime(final_df[time_column])

# Make sure it is in the right order
final_df = final_df.sort_values(time_column)

display(final_df)

### Make a plot: two data columns on different axes

Possible for two of the selected data columns. Even if more are given, only the first two in the list "selected_data_columns" will be used.

<mark>Note that only the latest year of data is plotted</mark>. This is because there are very many data points.


In [ ]:

time_column = 'TIMESTAMP'

# Select two of the data column in dataframe "final_df"
selected_data_columns = ['Temp [degC]', 'pCO2 [uatm]']

# Find the latest year based on the time_column
latest_year = final_df[time_column].dt.year.max()

# Filter the DataFrame to include only rows from the latest year
final_df_latest_year = final_df[final_df[time_column].dt.year == latest_year]

# Check if all selected columns are in renamed_data_columns
missing_columns = set(selected_data_columns) - set(available_data_columns)

if missing_columns or time_column not in final_df_latest_year.columns:
    print(f"One or more of the columns ({selected_data_columns}), or the time_column ({time_column}), are not in final_df.")
    print(f"Available columns are: {final_df_latest_year.columns}")  
else:
    # Set up the plot with the first variable
    fig, ax1 = plt.subplots()

    # Plot the first variable on the primary y-axis
    col_index_1 = available_data_columns.index(selected_data_columns[0])
    y_axis_label_1 = y_axis_labels[col_index_1]

    # 'b.' specifies that the data points should be blue and marked on the plot as circles.
    ax1.plot(final_df_latest_year[time_column], final_df_latest_year[selected_data_columns[0]], 'b.')
    ax1.set_xlabel('Time')
    ax1.set_ylabel(y_axis_label_1, color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    # Set the title with the station name
    ax1.set_title(station)

    if len(selected_data_columns) > 1:
        # Create a secondary y-axis for the second variable
        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

        # Plot the second variable on the secondary y-axis
        col_index_2 = available_data_columns.index(selected_data_columns[1])
        y_axis_label_2 = y_axis_labels[col_index_2]

        ax2.plot(final_df_latest_year[time_column], final_df_latest_year[selected_data_columns[1]], 'r.')
        ax2.set_ylabel(y_axis_label_2, color='r')
        ax2.tick_params(axis='y', labelcolor='r')

    # show the dates in this specific format (YYYY-MM-DD)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    
    # Rotate the dates to fit better
    ax1.tick_params(axis='x', rotation=45)

    # Add grid
    ax1.grid(True)

    # Show the plot
    fig.tight_layout()  # to make sure labels/axes don't overlap
    plt.show()
